In [3]:
import torch
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader,random_split
from __future__ import print_function
import numpy as np 
import pandas as pd
import os
import glob
import time
import random
import matplotlib.pyplot as plt 
from torch.autograd import Function
from collections import OrderedDict
import torch.nn as nn
import math
import copy
import torchvision.models as models
import pickle
import cv2
import wandb
from sklearn.model_selection import KFold

from datetime import timedelta, datetime

import decord
from decord import VideoReader
from decord import cpu, gpu

/home/anagara8/.conda/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
USE_DATASET = "preexposure"
percentage_chance = 0.01

In [5]:
base_path = '/media/data_cifs/nih/files_to_send/'
datasets = os.listdir(base_path)

postext_datasets = [dataset for dataset in datasets if 'postext' in dataset and "Trap" not in dataset ]
postret_datasets = [dataset for dataset in datasets if 'postret' in dataset and "Trap" not in dataset ]
postcond_datasets = [dataset for dataset in datasets if 'postcond' in dataset and "Trap" not in dataset ]
preexposure_datasets = [dataset for dataset in datasets if 'preexposure' in dataset and "Trap" not in dataset ]

if USE_DATASET == "postext":
    dataset_to_use = postext_datasets
elif USE_DATASET == "postret":
    dataset_to_use = postret_datasets
elif USE_DATASET == "preexposure":
    dataset_to_use = preexposure_datasets
else:
    dataset_to_use = postcond_datasets

all_datasets = [[('FC-A', 'preexposure', 'Trap'), ('FC-B','preexposure'), ('FC-C', 'preexposure'), ('FC-D', 'preexposure')],
            [('FC-A', 'postcond', 'Trap'), ('FC-B','postcond'), ('FC-C', 'postcond'), ('FC-D', 'postcond')],
            [('FC-A', 'postret', 'Trap'), ('FC-B','postret'), ('FC-C', 'postret'), ('FC-D', 'postret')],
            [('FC-A', 'postext', 'Trap'), ('FC-B','postext'), ('FC-C', 'postext'), ('FC-D', 'postext')]]


In [6]:
for folder in dataset_to_use:
    folder_path = base_path+folder
#     print("FOLDER:", folder)

    append_character = folder.split('_')[0][-1]
    append_character = "_"+append_character.lower()

    print(folder, ":", append_character)

    # Working with postcond FC-A:
    files = glob.glob(folder_path+"/*.mp4")
    files.sort()
    print("Number of files in Current Cohort (1 file = 1 hour of data):", len(files))

    # Sorting the file list to get data in order
    files = sorted(files)

    for file in files:     
        file_save_name = file.rfind("/", 0, file.rfind("/"))
        file_save_name = file[file_save_name:]
        file_save_name = file_save_name.replace("/", "")
        file_save_name = file_save_name.replace(".mp4", "")
        
#         print(file_save_name)
        tic = time.perf_counter()
        
        video_data = VideoReader(file, ctx=cpu())
        decord.bridge.set_bridge('torch')
        video_length = len(video_data)
        
        randomly_sampled_frames = [video_data[idx].numpy() for idx in np.random.randint(0, video_length, int(video_length*percentage_chance))] #random.choices(video_data, k=int(video_length*percentage_chance))
        np.save("./"+USE_DATASET+"/"+file_save_name, randomly_sampled_frames)
        
        toc = time.perf_counter()
        print("Completed", file_save_name, "in", int(toc - tic), "seconds.")
        break
    break

FC-D_preexposure_W1_05-30-2019 : _d
Number of files in Current Cohort (1 file = 1 hour of data): 144
FC-D_preexposure_W1_05-30-2019video_2019Y_05M_30D_10h_47m_41s_cam_17202339-0000
Completed FC-D_preexposure_W1_05-30-2019video_2019Y_05M_30D_10h_47m_41s_cam_17202339-0000 in 110 seconds.
